In [1]:
from langchain_community.document_loaders import TextLoader

In [2]:
temp_text_loader = TextLoader('./files/speech.txt')
temp_text_loader.load()

[Document(metadata={'source': './files/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\nâ€¦\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and f

I was getting the error for encoding so i first find the encoding and then specified to text loader.

In [3]:
from langchain_community.document_loaders import TextLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chardet
import re
import os

def chunk_generator(file):

    if not os.path.exists(file):
        raise FileNotFoundError(f"The file {file} does not exist.")

    split = os.path.splitext(file)
    file_type = split[1].lower()

    # Find the encoding type as sometimes it can't find automatically and needed for loading the data
    encoding_type = None
    if file_type != '.pdf':
        try:
            with open(file, 'rb') as f:
                result = chardet.detect(f.read())
                encoding_type = result['encoding']
        except Exception as e:
            raise RuntimeError(f"Error detecting encoding for {file}: {str(e)}")

    try:
        if file_type == '.pdf':
            document = PyPDFLoader(file).load()
            # OR
            # document = PyPDFLoader(file)
            # document.load()
        else:
            document = TextLoader(file, encoding = encoding_type).load()

    except Exception as e:
        raise RuntimeError(f"Error loading {file}: {str(e)}")
    
    for chunk in document:
        chunk.page_content = re.sub(r'\s+', ' ', chunk.page_content).strip()
        chunk.page_content = re.sub(r'-+', '-', chunk.page_content).strip()
        

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200
    )
    splitted_chunks = text_splitter.split_documents(document)

    print(f"Number of chunks: {len(splitted_chunks)}")
    print(f"First two chunks: {splitted_chunks[:2]}")
    return splitted_chunks

In [6]:
file1 = './files/software_engineering.mdx'
file2 = './files/Cryptography and Network Security.pdf'

splitted_chunks_1 = chunk_generator(file1)
splitted_chunks_2 = chunk_generator(file2)

Number of chunks: 257
First two chunks: [Document(metadata={'source': './files/software_engineering.mdx'}, page_content='- title: Software Engineering date: "2023-12-24T00:00:00Z" modifiedTime: "2023-12-27T02:00:00Z" tags: ["Computer Engineering", "Semester 5", "Software Engineering"] summary: - ## Three Marks ### What is Agile Manifesto? - The Agile Manifesto is a foundational document that outlines the core values and principles of agile software development. It serves as a guiding philosophy for agile methodologies in software development. The manifesto was created by a group of software developers who gathered at the Snowbird ski resort in Utah in 2001. - The Agile Manifesto consists of four core values and twelve guiding principles - Absolutely, let\'s break down the Agile Manifesto values and principles into simpler terms: **Agile Manifesto Values:** 1. People First: 2. Working Software Priority: 3. Customer Collaboration: 4. Adapting to Change: **Agile Manifesto Principles:** 1.

Because i do not open ai paid version free api always shown limit exceeded so i use huggingface embeding as it do not reuqire any api implimentation.

In [7]:
# # # from langchain_openai import OpenAIEmbeddings
# # # from langchain_google_vertexai import VertexAIEmbeddings
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
# # # from dotenv import load_dotenv

# # # load_dotenv()
# # # os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')


# db = Chroma.from_documents(
#     splitted_chunks_1,
#     HuggingFaceEmbeddings(),
#     persist_directory="./chroma_db/se_vector",
# )

In [13]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

def vector_generator(splitted_chunks, db_name):
    db_name = './chroma_db/' + str(db_name)
    db = Chroma.from_documents(
        splitted_chunks,
        HuggingFaceEmbeddings(),
        persist_directory = db_name
    )

    return db

In [9]:
db_name = "se_vector"
db1 = vector_generator(splitted_chunks_1, db_name)

C:\Users\Smit Vaghasiya\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
query = "What is Requirement Engineering? "
retrieved_results = db1.similarity_search(query)
for chunk in retrieved_results[:5]:
    print(chunk.page_content)

during verification and validation processes: - Black-box testing, white-box testing, integration testing, system testing, acceptance testing, etc. - Uncover defects and ensure the software meets specified requirements and user expectations. ### What is Requirement Engineering? How it is carried out in a Software Organizations? Requirement Engineering involves gathering, analyzing, documenting, and managing requirements for a software system. In software organizations, Requirement Engineering is conducted through several key steps: 1. **Elicitation:** Gathering requirements by interacting with stakeholders, users, and customers to comprehend their needs. 2. **Analysis:** Understanding and examining collected requirements to ensure they are consistent, complete, and feasible. 3. **Specification:** Documenting requirements in a clear and understandable manner, often using formats like use cases, user stories, or requirement documents. 4. **Validation:** Verifying if the specified


In [11]:
db_name = "cn_vector"
db2 = vector_generator(splitted_chunks_2, db_name)

In [20]:
query = "The Challenges of Computer Security"
retrieved_results = db2.similarity_search(query)
for chunk in retrieved_results[:5]:
    print(chunk.page_content)

7 The papers to read are [WARE79], [BROW72], [SALT75], [SHAN77], and [SUMM84]. Two more recent, short treatments of computer security are [ANDR04] and [LAMP04]. [NIST95] is an exhaustive (290 pages) treatment of the subject. Another good treatment is [NRC91]. Also useful is [FRAS97]. 7These classic papers are available in the Premium Content Web site for this book. ANDR04 Andrews, M., and Whittaker, J. “Computer Security.” IEEE Security and Privacy, September/October 2004. BROW72 Browne, P. “Computer Security—A Survey.” ACM SIGMIS Database, Fall 1972. FRAS97 Fraser, B. Site Security Handbook. RFC 2196, September 1997. LAMP04 Lampson, B. “Computer Security in the Real World,” Computer, June 2004. NIST95 National Institute of Standards and Technology. An Introduction to Computer Security: The NIST Handbook. Special Publication 800–12, October 1995. NRC91 National Research Council. Computers at Risk: Safe Computing in the Information Age. Washington, D.C.: National Academy Press, 1991.
fa

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os

def chat_module(query):
    load_dotenv()
    os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

    db_name = "cn_vector"
    db = Chroma(
        persist_directory=f"./chroma_db/{db_name}",
        embedding_function=HuggingFaceEmbeddings()
    )

    llm = ChatGroq(
        model_name="llama3-8b-8192",
        temperature=0.5,
        max_tokens=1000
    )

    prompt_template = """
    You are an AI assistant designed to answer questions based solely on the provided context.
    Do not include information outside the given context. If the context is not relevant to the question, 
    state that no relevant information is available.And explain this topic in the full detail acorrding the context given.
    Context: {context}
    Question: {question}
    Answer:
    """

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=db.as_retriever(search_kwargs={"k": 5}),
        chain_type_kwargs={"prompt": prompt}
    )

    result = qa_chain.run(query)
    return result

In [14]:
query = input("Ask me any question related to cryptography you want to know about: ")
print(chat_module(query))

Based on the provided context, I will provide a detailed explanation of cryptography and its uses.

**What is Cryptography?**

Cryptography is the practice and study of techniques for secure communication by transforming plaintext (readable data) into ciphertext (unreadable data) to prevent unauthorized access. It involves the use of algorithms and protocols to encrypt and decrypt data, ensuring the confidentiality, integrity, and authenticity of the information.

**Uses of Cryptography**

1. **Encryption**: Cryptography is used to encrypt data to prevent unauthorized access. This is done by using algorithms and keys to transform plaintext into ciphertext.
2. **Decryption**: Cryptography is also used to decrypt data, allowing authorized parties to access the original plaintext.
3. **Authentication**: Cryptography is used to verify the authenticity of data, ensuring that it has not been tampered with or altered during transmission.
4. **Integrity**: Cryptography is used to ensure the in